In [13]:
import os
import sys
import anthropic
# import ollama
import random
import pandas as pd
from tqdm import tqdm
from google.generativeai.types import RequestOptions
from google.api_core import retry
from typing import List, Tuple
import json
from openai import OpenAI
import datetime

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [10]:
base_prompt = """
# General Instructions
You are a helpful assistant tasked with highlighting key facts in text. Given a question and answer pair, you need to identify distinct sections of the text that were needed to answer the question. You are only concerned with identifying excerpts of the text that are ESSENTIAL to answering the question. You should ignore irrelevant information that does not help answer the final question. You should extract the exact parts of the text, not summarized versions of the text. After you have decided what specific quotes to use, tag the original question and answer with xml tags around those quotes. The xml tags should follow the format of <fact1>, <fact2>, etc. Here is what your response should look like:

### Fact Extraction Reasoning:
.... (this is your scratch pad to decide which information is most important)

### Reformatted Text
...

# Fact Extraction Details
Each block of text has a large amount of information that could be a valid fact. However, you should only extract the most important quotes. Try to keep each quote as short as possible while still maintaining the essential information.

## Properly Done Example
Here is an example of properly extracting the information needed to answer the final question:

### Original Text:
Question: The following paragraphs each describe a set of seven objects arranged in a fixed order. The statements are logically consistent within each paragraph. In an antique car show, there are seven vehicles: a bus, a motorcyle, a hatchback, a station wagon, a minivan, a truck, and a limousine. The station wagon is the fourth-newest. The motorcyle is newer than the truck. The station wagon is older than the hatchback. The minivan is newer than the hatchback. The bus is newer than the minivan. The truck is newer than the limousine.
Options:
(A) The bus is the third-oldest
(B) The motorcyle is the third-oldest
(C) The hatchback is the third-oldest
(D) The station wagon is the third-oldest
(E) The minivan is the third-oldest
(F) The truck is the third-oldest
(G) The limousine is the third-oldest

Answer: To determine which vehicle is the third-oldest, we need to analyze the given facts:
1. **The station wagon is the fourth-newest**: This means there are three vehicles that are newer than the station wagon.
2. **The motorcycle is newer than the truck**: This indicates that the motorcycle is not the oldest and is positioned above the truck in terms of age.
3. **The station wagon is older than the hatchback**: This means the hatchback is newer than the station wagon.
4. **The minivan is newer than the hatchback**: This places the minivan above the hatchback in terms of age.
5. **The bus is newer than the minivan**: This means the bus is the newest among the minivan and the hatchback.
6. **The truck is newer than the limousine**: This indicates that the limousine is older than the truck.
Now, let's summarize the order based on the information:
- Since the station wagon is the fourth-newest, we can denote the order as follows:
  - 1st: Bus (newest)
  - 2nd: Minivan (newer than hatchback)
  - 3rd: Motorcycle (newer than truck)
  - 4th: Station Wagon (given)
  - 5th: Hatchback (older than station wagon)
  - 6th: Truck (newer than limousine)
  - 7th: Limousine (oldest)
From this arrangement, we can see that the third-oldest vehicle is the **motorcycle**. 
The answer is {B}.

### Fact Extraction Reasoning:
The question asks us to determine the third-oldest vehicle in the list. To answer this question, we need all the relative ages of the vehicles. I'll focus on just taging the specific ages of each of the vehicels in the text.

### Reformatted Text:
Question: The following paragraphs each describe a set of seven objects arranged in a fixed order. The statements are logically consistent within each paragraph. In an antique car show, there are seven vehicles: a bus, a motorcyle, a hatchback, a station wagon, a minivan, a truck, and a limousine. The <fact1>station wagon is the fourth-newest</fact1>. The <fact2>motorcyle is newer than the truck</fact2>. The <fact3>station wagon is older than the hatchback</fact3>. The <fact4>minivan is newer than the hatchback</fact4>. The <fact5>bus is newer than the minivan</fact5>. The <fact6>truck is newer than the limousine</fact6>.
Options:
(A) The bus is the third-oldest
(B) The motorcyle is the third-oldest
(C) The hatchback is the third-oldest
(D) The station wagon is the third-oldest
(E) The minivan is the third-oldest
(F) The truck is the third-oldest
(G) The limousine is the third-oldest

Answer: To determine which vehicle is the third-oldest, we need to analyze the given facts:
1. **<fact1>The station wagon is the fourth-newest</fact1>**: This means there are three vehicles that are newer than the station wagon.
2. **<fact2>The motorcycle is newer than the truck</fact2>**: This indicates that the motorcycle is not the oldest and is positioned above the truck in terms of age.
3. **<fact3>The station wagon is older than the hatchback</fact3>**: This means the hatchback is newer than the station wagon.
4. **<fact4>The minivan is newer than the hatchback</fact4>**: This places the minivan above the hatchback in terms of age.
5. **<fact5>The bus is newer than the minivan</fact5>**: This means the bus is the newest among the minivan and the hatchback.
6. **<fact6>The truck is newer than the limousine</fact6>**: This indicates that the limousine is older than the truck.
Now, let's summarize the order based on the information:
- Since the <fact1>station wagon is the fourth-newest</fact1>, we can denote the order as follows:
  - 1st: Bus (newest)
  - 2nd: Minivan (newer than hatchback)
  - 3rd: Motorcycle (newer than truck)
  - 4th: Station Wagon (given)
  - 5th: Hatchback (older than station wagon)
  - 6th: Truck (newer than limousine)
  - 7th: Limousine (oldest)
From this arrangement, we can see that the third-oldest vehicle is the **motorcycle**. 
The answer is {B}.

## What NOT to Extract
Given an inputted text, there are large amounts of possible facts that could be used. However, not all of these citations are relevant to the final question. You should only extract information that is essential to answering the question. Here is an example of a poorly tagged response that has too many extracted facts:

### Original Text:
Question: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?

Answer: Betty currently has half of the $100 she needs, which is $50. Her parents give her $15, and her grandparents give her twice that amount, which is $30. Adding the $15 from her parents and $30 from her grandparents to the $50 she already has, Betty now has $50 + $15 + $30 = $95. Since the wallet costs $100, she still needs $100 - $95 = $5 more to buy the wallet.

### Fact Extraction Reasoning:
...

### Reformatted Text:
Question: <fact1>Betty is saving money</fact1> for a <fact2>new wallet</fact2> which costs <fact3>$100</fact3>. <fact4>Betty has only half</fact4> of the <fact5>money she needs</fact5>. <fact6>Her parents decided to give</fact6> her <fact7>$15 for that purpose</fact7>, and <fact8>her grandparents twice as much as her parents</fact8>. <fact9>How much more money</fact9> <fact10>does Betty need to buy the wallet</fact10>?

Answer: <fact4>Betty currently has half</fact4> of the <fact3>$100</fact3> she needs, which is $50. <fact6>Her parents give her</fact6> <fact7>$15</fact7>, and <fact8>her grandparents give her twice that amount</fact8>, which is $30. Adding the <fact7>$15</fact7> from her parents and <fact8>$30</fact8> from her grandparents to the <fact4>$50 she already has</fact4>, Betty now has <fact4>$50</fact4> + <fact7>$15</fact7> + <fact8>$30</fact8> = $95. Since the wallet costs <fact3>$100</fact3>, she still needs <fact3>$100</fact3> - $95 = $5 more to buy the <fact2>wallet</fact2>.

While all of these tags do properly wrap the quotes from the original text, they are not all necessary to answer the question. Tags like <fact1>, <fact2>, <fact5>, etc contain information that is not essential to answering the final question. Additionally, the tags should be as concise as possible while still providing the necessary information. Many of these tags are redundant or overly verbose, making the response less clear and concise.

**Tag the following question:**
"""

In [11]:
question = """
Question: We have three blocks, A, B and C. Block A has a medium blue square. Below block A is block B which has one medium black square. To the left of block B there is block C which has two medium blue squares. Medium blue square number one is below medium blue square number two. A medium yellow square is below medium blue square number two and medium blue square number one. What is to the left of the black thing? a medium blue square that is in block A or a medium blue square number two?
(a) medium blue square  that is in block A
(b) medium blue square  number two
(c) both of them
(d) none of them
Answer: Block A has a medium blue square, and Block B (below A) has a black square. Block C, to the left of Block B, has two medium blue squares, with blue square number two on top and number one below it. Since the blue squares in Block C are directly to the left of the black square in Block B.
The answer is {B}.

"""

In [12]:
prompt = base_prompt + question
print(prompt)


# General Instructions
You are a helpful assistant tasked with highlighting key facts in text. Given a question and answer pair, you need to identify distinct sections of the text that were needed to answer the question. You are only concerned with identifying excerpts of the text that are ESSENTIAL to answering the question. You should ignore irrelevant information that does not help answer the final question. You should extract the exact parts of the text, not summarized versions of the text. After you have decided what specific quotes to use, tag the original question and answer with xml tags around those quotes. The xml tags should follow the format of <fact1>, <fact2>, etc. Here is what your response should look like:

### Fact Extraction Reasoning:
.... (this is your scratch pad to decide which information is most important)

### Reformatted Text
...

# Fact Extraction Details
Each block of text has a large amount of information that could be a valid fact. However, you should on

In [15]:
def query_4o(prompt: str) -> str:
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        temperature=0
    )

    return completion.choices[0].message.content

print(query_4o(prompt))

### Fact Extraction Reasoning:
To determine what is to the left of the black square, we need to focus on the relative positions of the blocks and the squares within them. The key information is the position of Block C relative to Block B and the arrangement of the squares within Block C.

### Reformatted Text
Question: We have three blocks, A, B and C. Block A has a medium blue square. Below block A is block B which has <fact1>one medium black square</fact1>. To the left of block B there is block C which has <fact2>two medium blue squares</fact2>. Medium blue square number one is below medium blue square number two. A medium yellow square is below medium blue square number two and medium blue square number one. What is to the left of the black thing? a medium blue square that is in block A or a medium blue square number two?
(a) medium blue square that is in block A
(b) medium blue square number two
(c) both of them
(d) none of them
Answer: Block A has a medium blue square, and Block B

In [ ]:
import re
import os

def add_color_to_tags_new(text):
    """
    This function finds all unique tags in the text and assigns each a color from a predefined palette.
    It then replaces the tags with styled <span> elements that include the assigned background color.
    """
    # Find all unique tags in the text using regex
    tags = set(re.findall(r'<([A-Za-z]+\d*)>', text))

    # Predefined color palette
    color_palette = [
        'lightyellow', 'lightblue', 'lightgreen', 'lightcoral',
        'lightcyan', 'lightpink', 'lightsalmon', 'lightgray',
        'lightgoldenrodyellow', 'lightseagreen', 'lightskyblue',
        'lightsteelblue',
        'lavender', 'peachpuff', 'paleturquoise', 'wheat', 'mistyrose'
    ]

    # Dictionary to hold tag-color mapping
    tag_color_mapping = {}

    # Assign colors to tags, cycling through the color palette if necessary
    for i, tag in enumerate(sorted(tags)):
        color = color_palette[i % len(color_palette)]
        tag_color_mapping[tag] = color

    # Function to replace tags with styled spans including class names
    def replace_tag(match):
        tag = match.group(1)
        content = match.group(2)
        color = tag_color_mapping.get(tag, 'lightgray')  # Default color if not found
        return f'<span class="{tag}" style="background-color: {color}; font-weight: bold;">{content}</span>'

    # Regex to find tags and replace them with styled spans
    tag_regex = re.compile(r'<([A-Za-z]+\d*)>\s*([\s\S]*?)\s*</\1>')

    # Replace all tags with styled spans
    text = tag_regex.sub(replace_tag, text)

    return text


def create_highlight_html(questions, output_file='questions_visualization.html'):
    """
    This function takes a list of question strings, applies color highlighting to any tags within them,
    and generates an HTML file to display the visualized questions.
    """
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Questions Visualization</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f9f9f9;
            }
            .container {
                background-color: #ffffff;
                padding: 15px 20px;
                margin-bottom: 15px;
                border-radius: 6px;
                box-shadow: 0 1px 3px rgba(0,0,0,0.1);
            }
            .question {
                font-size: 1.1em;
                line-height: 1.5;
            }
            /* Styles for the highlighted spans */
            .highlighted {
                padding: 2px 4px;
                border-radius: 3px;
                display: inline-block;
            }
        </style>
    </head>
    <body>
    <h1>Questions Visualization</h1>
    <div class="summary">
        <strong>Total Questions:</strong> {total_questions}
    </div>
    <hr>
    """

    # Process each question and append to HTML content
    for idx, question in enumerate(questions, 1):
        try:
            # Apply color to tags in the question
            highlighted_question = add_color_to_tags_new(question)
            
            # Replace newline characters with <br> tags to ensure they are rendered in HTML
            highlighted_question = highlighted_question.replace('\n', '<br>')

            # Build the HTML structure for each question
            html_content += f"<div class='container'>"
            html_content += f"<div class='question'><strong>Question {idx}:</strong> {highlighted_question}</div>"
            html_content += "</div>\n"
        except Exception as e:
            print(f"Cannot process question {idx}: {e}")
            continue

    # Close the HTML tags
    html_content += """
    </body>
    </html>
    """

    # Insert the total number of questions into the summary section
    html_content = html_content.format(total_questions=len(questions))

    # Write the HTML content to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(html_content)

    print(f"HTML content has been successfully written to {output_file}")


def main():
    """
    Main function to execute the visualization process.
    """
    # Define your array of string questions here
    questions = [
        """### Fact Extraction Reasoning:
To determine what is to the left of the black square, we need to focus on the relative positions of the blocks and the squares within them. The key information is the position of Block C relative to Block B and the arrangement of the squares within Block C.

### Reformatted Text
Question: We have three blocks, A, B and C. Block A has a medium blue square. Below block A is block B which has <fact1>one medium black square</fact1>. To the left of block B there is block C which has <fact2>two medium blue squares</fact2>. Medium blue square number one is below medium blue square number two. A medium yellow square is below medium blue square number two and medium blue square number one. What is to the left of the black thing? a medium blue square that is in block A or a medium blue square number two?
(a) medium blue square that is in block A
(b) medium blue square number two
(c) both of them
(d) none of them
Answer: Block A has a medium blue square, and Block B (below A) has a <fact1>black square</fact1>. Block C, <fact3>to the left of Block B</fact3>, has <fact2>two medium blue squares</fact2>, with blue square number two on top and number one below it. Since the blue squares in Block C are directly to the left of the black square in Block B.
The answer is {B}.""",
    ]

    # Specify the output HTML file name
    output_html_file = 'questions_visualization.html'

    # Generate the HTML visualization
    create_highlight_html(questions, output_html_file)


if __name__ == "__main__":
    main()
